<a href="https://colab.research.google.com/github/nat-smithh/ag-sci/blob/main/sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# analyzing sequence data

In this jupyter notebook, we'll compare a few different approaches for analyzing sequence data.

Of course, we'll need some sequence data to analyze.

In tensorflow, a 'sequence' is a 1-dimensional 'list' of data, with each 'data point' being a vector. So:

    [ 1.2, 2.3, 3.4, 4.5 ]

is *not* a sequence in tensorflow, but:

    [ [1.2], [2.3], [3.4], [4.5] ]

*is* a sequence.

You can think of a 'sequence' as similar to a 2-dimensional 'image' in tensorflow, except you are stuck in 1-dimension, in the case of sequence data. Similar to the case of 'image' data, we can discuss the concept of 'channels' in sequence data.

The sequence above has a single 'channel', whereas the sequence below has 3 'channels':

    [ [1,2,3], [4,5,6], [7,8,9], [10,11,12] ]

As in the case of 2D 'image' data, the *meaning* of the channels will depend on the *meaning* of the data, which is particular to how the data were collected and what the analysis problem is trying to accomplish.

For this exercise, we'll just simulate some sequence data, so we can gain some experience analyzing such data and compare different models.

In this case, we'll use the scikit-learn make_classification function to simulate sequences of 128 data points (or 'features'), each of which is one-dimensional. For classification, we'll set 32 features as 'informative' or correlated with class labels, while the remaining features in the sequence are 'random' with regard to class labels (ie, not informative). We'll simulate a total of 38,262 data samples and use binary class labels.

In [1]:
import numpy as np
import tensorflow as tf
import sklearn.datasets
import sklearn.model_selection


x, y = sklearn.datasets.make_classification(n_samples=38262,
                                            n_features=128,
                                            n_informative=32,
                                            random_state=8792439)
train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(x,y, test_size=0.2, random_state=849691)
train_x = np.expand_dims(train_x, axis=-1)
valid_x = np.expand_dims(valid_x, axis=-1)
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)
print(train_data, valid_data)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In this case, we used the scikit-learn function train_test_split to randomly divide our simulated data into 80% training and 20% validation data.

Also notice that we had to use the numpy expand_dims function to transform our simulated data into the correct 'shape' for tensorflow sequences. The scikit-learn make_classification function creates 'sequences' of the form:

    [ a, b, c, d ]

which is *not* a tensorflow sequence, as it lacks the 'channel dimension' (or rank). To fix this, we 'expand' the last dimension, so the data takes on the form:

    [ [a], [b], [c], [d] ]

which is compatible with how tensorflow sequence models work.

After getting our data in the correct format for tensorflow, it's easy to create tensorflow Dataset objects using the from_tensor_slices function.

You should see that your Dataset objects have the shape:

    ((None, 128,1), (None,))

So, the explanatory variables (ie, the sequence data) have shape:

    (None, 128, 1)

That is, there are 128 'features' in the sequence, and each feature has dimensionality 1. Another way of saying this is that the sequences have *length* 128, and the sequence data has a single *channel*.

## convolution model

First, we'll analyze our sequence data using a convolution neural-network model, similar to what we did with the 2-dimensional 'image' data.

The only 'difference' we need to be aware of when applying convolution models to sequence data is that we'll need to use a tf.keras.layers.Conv1D object to model the 1-dimensional sequence data, rather than the tf.keras.layers.Conv2D object we used to model 2-dimensional 'image' data.

For the 1D convolutions, there is only a single 'rank' in the kernel_size, so we'll need to set:

    kernel_size=(3,)

when we create the 1D convolution layer. Of course, we could use a larger kernel size, if we wanted to.

For now, we'll just use a single Conv1D layer with ReLU activation in our model. We'll need to decide on the number of filters we want to use in our convolution layer. For now, we'll just set:

    filters=64

as a reasonable starting value.

As with the 2D 'image' data, we'll need to 'flatten' our sequence data to remove the 'channel' dimension, before sending it to the classification layer.

This is a binary classification problem, so we'll use a Dense layer with a single unit and sigmoid activation as the 'decision layer' in the network.

Finally, we'll need to use binary crossentropy loss as our loss function. In this example, we'll use the Adam optimizer, and we'll make sure to track the 'accuracy' of our model as it trains, which is commonly done for classification problems, as humans tend to 'reason' better with accuracy than with loss outputs.

With such a simple model, we should get a reasonable model fit after 50 epochs of training using the Adam optimizer. You might want to make sure you're running the model fit with GPU hardware acceleration, though. Click on the downward-arrow in the upper-right corner; select 'View resources', then click on 'Change Runtime Type' and select GPU hardware acceleration before saving. Then you can run the following code cell.

Training should take about 5 seconds / epoch with GPU acceleration. So, about 5 minutes or so.

In [2]:
import numpy as np
import tensorflow as tf
import sklearn.datasets
import sklearn.model_selection

# simulate and package data
x, y = sklearn.datasets.make_classification(n_samples=38262,
                                            n_features=128,
                                            n_informative=32,
                                            random_state=8792439)
train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(x,y, test_size=0.2, random_state=849691)
train_x = np.expand_dims(train_x, axis=-1)
valid_x = np.expand_dims(valid_x, axis=-1)
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)
print(train_data, valid_data)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=(3,), activation=tf.keras.activations.relu, input_shape=(128,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 126, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8064)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         8,065 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,321 (32.50 KB)

 Trainable params: 8,321 (32.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8598 - loss: 0.3227 - val_accuracy: 0.8904 - val_loss: 0.2765
Epoch 2/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8928 - loss: 0.2623 - val_accuracy: 0.8888 - val_loss: 0.2785
Epoch 3/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8968 - loss: 0.2571 - val_accuracy: 0.8889 - val_loss: 0.2796
Epoch 4/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8987 - loss: 0.2529 - val_accuracy: 0.8900 - val_loss: 0.2791
Epoch 5/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8998 - loss: 0.2490 - val_accuracy: 0.8910 - val_loss: 0.2788
Epoch 6/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9008 - loss: 0.2458 - val_accuracy: 0.8902 - val_loss: 0.2784
Epoch 7/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9013 - loss: 0.2433 - val_accuracy: 0.8904 - val_loss: 0.2781
Epoch 8/50
957/957 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9017 - loss: 0.2414 - val_accuracy: 0.

This 1D convolution network has 8321 trainable parameters, the vast majority of which (8065) are in the final decision layer.

There are only 256 trainable parameters in the convolution layer of the network. We defined our convolution kernel size as (3,), so each convolution neuron has 3 input weights, plus a bias term, so there are 3+1=4 trainable parameters per convolution 'filter'. We wanted 64 filters, so there are 64*4=256 trainable parameters in the convolution layer.

After training for 50 epochs using Adam optimization, my convolution network was able to achieve 0.91 classification accuracy on the training data and 0.89 classification accuracy on the validation data. So, about 90% of the sequences are correctly classified by this model.

Also note how long it takes to train this model. In my case, each epoch took about 4 seconds to train using GPU hardware acceleration.

## simple Recurrent Neural Network (RNN) model

Next, let's compare the convolution model to what we'd get if we analyzed our data using a simple recurrent neural network model (aka, "RNN").

Tensorflow implements a simple RNN using a tf.keras.layers.SimpleRNN object, which we can add to our Sequential model just like any other Layer object.

There are a couple options to the SimpleRNN that we'll want to set. First, we'll want to set the number of RNN "units" in the layer. In the Conv1D model, we used 64 'filters'. In the SimpleRNN model, we'll use *half* that number, or 32 'units' in the RNN:

    units=32

This will result in an RNN model with slightly *fewer* trainable parameters, compared to the Conv1D model we used previously.

We'll also want to set the:

    return_sequences=True

option flag. In the default behaviour, the SimpleRNN object *only* returns the output from the *last* neuron in the sequence. By setting return_sequences=True, we'll get the outputs from *all* the neurons in the sequence layer, so the decision layer will have more information from which to classify the sequences.

Other than that, we'll just need to make sure we set the input_shape option, as the SimpleRNN layer will be the first layer in the Sequential model.

The SimpleRNN model's compilation and fitting is implemented in exactly the same way as in the case of the convolution model.

By the way, you should be prepared to have something else to do for awhile once you start the model fitting process in the following code cell. Remember how we discussed the sequential nature of RNN models, and how they take longer to train... In fact, the RNN model is so slow that we're only going to train it for 20 epochs.

In [4]:
import numpy as np
import tensorflow as tf
import sklearn.datasets
import sklearn.model_selection

# simulate and package data
x, y = sklearn.datasets.make_classification(n_samples=38262,
                                            n_features=128,
                                            n_informative=32,
                                            random_state=8792439)
train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(x,y, test_size=0.2, random_state=849691)
train_x = np.expand_dims(train_x, axis=-1)
valid_x = np.expand_dims(valid_x, axis=-1)
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)
print(train_data, valid_data)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(units=32, activation=tf.keras.activations.relu, input_shape=(128,1), return_sequences=True))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 128, 32)        │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,185 (20.25 KB)

 Trainable params: 5,185 (20.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.8499 - loss: 0.3410 - val_accuracy: 0.9041 - val_loss: 0.2439
Epoch 2/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9095 - loss: 0.2250 - val_accuracy: 0.9125 - val_loss: 0.2300
Epoch 3/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9186 - loss: 0.2084 - val_accuracy: 0.9144 - val_loss: 0.2235
Epoch 4/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9211 - loss: 0.1982 - val_accuracy: 0.9164 - val_loss: 0.2180
Epoch 5/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9246 - loss: 0.1891 - val_accuracy: 0.9191 - val_loss: 0.2132
Epoch 6/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9297 - loss: 0.1809 - val_accuracy: 0.9191 - val_loss: 0.2105
Epoch 7/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9324 - loss: 0.1733 - val_accuracy: 0.9213 - val_loss: 0.2069
Epoch 8/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9357 - loss: 0.1670 - val_accurac

Make sure you keep track of the model's number of parameters, how long it took to train, and the final accuracy and loss values, for the associated quiz!

## Long Short-Term Memory (LSTM) model

Next, let's compare a similar LSTM model to the SimpleRNN model we just implemented.

Tensorflow implements LSTM neurons as the object, tf.keras.layers.LSTM, so all we need to do is to replace the code:

    tf.keras.layers.SimpleRNN(...)

With:

    tf.keras.layers.LSTM(...)

in the above code cell, and we'll have a similar model that uses LSTM neurons. We can keep all the other parameters and options the same as in the previous code cell.

Implement the LSTM model below, and run the following code cell to train your LSTM network.

In [6]:
import numpy as np
import tensorflow as tf
import sklearn.datasets
import sklearn.model_selection

# simulate and package data
x, y = sklearn.datasets.make_classification(n_samples=38262,
                                            n_features=128,
                                            n_informative=32,
                                            random_state=8792439)
train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(x,y, test_size=0.2, random_state=849691)
train_x = np.expand_dims(train_x, axis=-1)
valid_x = np.expand_dims(valid_x, axis=-1)
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)
print(train_data, valid_data)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=20, activation=tf.keras.activations.relu, input_shape=(128,1), return_sequences=True))
model.add(tf.keras.layers.Flatten()) # Add Flatten layer to reduce dimensions
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid)) # Add Dense layer for classification
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128, 20)        │         1,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         2,561 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,321 (16.88 KB)

 Trainable params: 4,321 (16.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - accuracy: 0.7524 - loss: 23.8234 - val_accuracy: 0.8406 - val_loss: 0.3911
Epoch 2/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8449 - loss: 0.3801 - val_accuracy: 0.8544 - val_loss: 0.3601
Epoch 3/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.8552 - loss: 0.3508 - val_accuracy: 0.8585 - val_loss: 0.3431
Epoch 4/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8622 - loss: 0.3340 - val_accuracy: 0.8637 - val_loss: 0.3311
Epoch 5/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8683 - loss: 0.3215 - val_accuracy: 0.8689 - val_loss: 0.3213
Epoch 6/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8722 - loss: 0.3118 - val_accuracy: 0.8714 - val_loss: 0.3140
Epoch 7/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8746 - loss: 0.3044 - val_accuracy: 0.8730 - val_loss: 0.3111
Epoch 8/20
957/957 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8785 - loss: 0.2993 -

Again, keep track of the model's parameter count, training time and accuracy values for the associated quiz.

You may have noticed that the LSTM training time is *much less* than the training time required for the SimpleRNN model!

That's odd; I thought that LSTM neurons were *more* complicated than SimpleRNN neurons, and LSTM would therefore require *more* training time??

Well, in this case there are some back-end optimizations that significantly speed up LSTM training. In particular, specific combinations of options allow LSTM training to take advantage of NVIDIA's cuDNN library for neural network training, which is very fast. So long as you are happy with the default LSTM parameters, you can run LSTM training much faster than the simpler SimpleRNN model.